In [1]:
import time
import re
import datetime as dt
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

### import data

In [2]:
loc_PM = pd.read_csv('./META/pmmap.csv', index_col = 'Location', encoding = 'UTF-8')
del loc_PM['Description']
loc_PM = loc_PM[:4]
loc_PM

,Latitude,Longitude
Location,,
아름동,36.512252,127.246789
신흥동,36.592887,127.292550
노은동,36.368242,127.318498
문창동,36.317215,127.437825


In [3]:
loc_AWS = pd.read_csv('./META/awsmap.csv', index_col = 'Location', encoding = 'UTF-8')
loc_AWS.head()

,Latitude,Longitude,Description
Location,,,
오월드,36.2913,127.3959,대전광역시 중구사정공원로 70 오월드 내 플라워랜드
세천,36.3402,127.4938,대전광역시 동구세천동 63-1
장동,36.4135,127.4382,대전광역시 대덕구장동 360-2
세종고운,36.5315,127.2406,세종특별자치시 고운동산 25번지
세종금남,36.4585,127.2688,세종특별자치시 시군구금남면 성덕리 77-3


In [16]:
for nm_PM in loc_PM.index:
    globals()['PM_{}'.format(nm_PM)]=pd.read_csv(f'./TRAIN/{nm_PM}.csv',
                                                 encoding = 'UTF-8')
for nm_AWS in loc_AWS.index:
    globals()['AWS_{}'.format(nm_AWS)]=pd.read_csv(f'./TRAIN_AWS/{nm_AWS}.csv',
                                                 encoding = 'UTF-8')

### 내심, 무게중심 거리 구하는 함수

In [4]:
import math

def incenter(x_lst, y_lst):  # 내심
    p1_x = x_lst[0];p1_y = y_lst[0]
    p2_x = x_lst[1];p2_y = y_lst[1]
    p3_x = x_lst[2];p3_y = y_lst[2]
    l12 = math.sqrt((p1_x - p2_x)**2 + (p1_y - p2_y)**2)
    l13 = math.sqrt((p1_x - p3_x)**2 + (p1_y - p3_y)**2)
    l23 = math.sqrt((p2_x - p3_x)**2 + (p2_y - p3_y)**2)
    X = (l12*p3_x + l13*p2_x + l23*p1_x) / (l12 + l13 + l23)
    Y = (l12*p3_y + l13*p2_y + l23*p1_y) / (l12 + l13 + l23)
    return (X,Y)

def centroid(x_lst, y_lst):   # 무게중심
    n = len(x_lst)
    X = 0;Y = 0
    X = sum(x_lst);Y = sum(y_lst)
    X /= n;Y /= n
    return (X,Y)

def distance(p1, p2):  #거리
    p1_x = p1[0];p1_y = p1[1]
    p2_x = p2[0];p2_y = p2[1]
    D = math.sqrt((p1_x - p2_x)**2 + (p1_y - p2_y)**2)
    return D

### PM별 반경 이내 AWS 찾기

In [5]:
rads = []
Near_AWS = []
Count_AWS = []
lctn_AWS = {}
for i in range(len(loc_PM.index)):
    lat_PM = loc_PM['Latitude'][i]
    lon_PM = loc_PM['Longitude'][i]
    
    lats_AWS = [];lons_AWS = []
    
    near_AWS = ''
    rad = 0
    count_AWS = 0
    
    while len(near_AWS) == 0:
        rad += 0.15       # 4개 이상의 AWS 잡기 위하여
        #rad += 0.1            # 빈경을 0.1씩 늘려가며 search
        # 반경 0.05로 할 시 AWS 1개씩 대응되었음
        
        for j in range(len(loc_AWS.index)):
            lat_AWS = loc_AWS['Latitude'][j]
            lon_AWS = loc_AWS['Longitude'][j]
            if distance((lat_AWS, lon_AWS), (lat_PM, lon_PM)) < rad:
                count_AWS += 1
                lats_AWS.append(lat_AWS);lons_AWS.append(lon_AWS);
                near_AWS += str(',') + loc_AWS.index[j]
    
    rads.append(rad)
    Near_AWS.append(near_AWS[1:])
    Count_AWS.append(count_AWS)
    lctn_AWS[f'{loc_PM.index[i]}'] = [lats_AWS,lons_AWS]
    
loc_PM['Rad'] = rads
loc_PM['Near_AWS'] = Near_AWS
loc_PM['Count_AWS'] = Count_AWS
loc_PM

,Latitude,Longitude,Rad,Near_AWS,Count_AWS
Location,,,,,
아름동,36.512252,127.246789,0.15,"세종고운,세종금남,세종연서,공주",4
신흥동,36.592887,127.292550,0.15,"세종고운,세종금남,세종연서,세종전의",4
노은동,36.368242,127.318498,0.15,"오월드,장동,세종금남,계룡",4
문창동,36.317215,127.437825,0.15,"오월드,세천,장동",3


In [6]:
loc_AWS.loc['세종고운']

Latitude                   36.5315
Longitude                 127.2406
Description    세종특별자치시 고운동산 25번지  
Name: 세종고운, dtype: object

### Near_AWS 대표 좌표 구하기

In [7]:
cent_lat = [];cent_lon=[];distance_PC = []   # 일단 무게중심, 위도경도만
for nm_PM in loc_PM.index:
    cent_point = centroid(lctn_AWS[nm_PM][0], lctn_AWS[nm_PM][1])
    distance_PC.append(distance(cent_point,(loc_PM.loc[nm_PM]['Latitude'],loc_PM.loc[nm_PM]['Longitude'])))
    cent_lat.append(cent_point[0]);cent_lon.append(cent_point[1])
    
loc_PM['Cent_lat'] = cent_lat
loc_PM['Cent_lon'] = cent_lon
loc_PM['PM-Cent_d'] = distance_PC

loc_PM

,Latitude,Longitude,Rad,Near_AWS,Count_AWS,Cent_lat,Cent_lon,PM-Cent_d
Location,,,,,,,,
아름동,36.512252,127.246789,0.15,"세종고운,세종금남,세종연서,공주",4,36.509875,127.231625,0.015349
신흥동,36.592887,127.292550,0.15,"세종고운,세종금남,세종연서,세종전의",4,36.561875,127.247625,0.054589
노은동,36.368242,127.318498,0.15,"오월드,장동,세종금남,계룡",4,36.369125,127.335900,0.017425
문창동,36.317215,127.437825,0.15,"오월드,세천,장동",3,36.348333,127.442633,0.031488


In [31]:
print(sum(PM_아름동['PM2.5'].isnull()/len(PM_아름동['PM2.5'])))
print(sum(PM_신흥동['PM2.5'].isnull()/len(PM_신흥동['PM2.5'])))
print(sum(PM_노은동['PM2.5'].isnull()/len(PM_노은동['PM2.5'])))
print(sum(PM_문창동['PM2.5'].isnull()/len(PM_문창동['PM2.5'])))

0.022758384668035297
0.02164613278576294
0.02635181382614599
0.01813826146475012


In [33]:
AWS_세종고운.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   연도       35064 non-null  int64  
 1   일시       35064 non-null  object 
 2   지점       35064 non-null  object 
 3   기온(°C)   34764 non-null  float64
 4   풍향(deg)  34764 non-null  float64
 5   풍속(m/s)  34764 non-null  float64
 6   강수량(mm)  34764 non-null  float64
 7   습도(%)    34764 non-null  float64
dtypes: float64(5), int64(1), object(2)
memory usage: 2.1+ MB


In [62]:
spec = loc_PM.loc['아름동'][['Rad','Count_AWS','Cent_lat','Cent_lon','PM-Cent_d']]
PM_아름동[['Rad','Count_AWS','Cent_lat','Cent_lon','PM-Cent_d']] = loc_PM.loc['아름동'][['Rad','Count_AWS','Cent_lat','Cent_lon','PM-Cent_d']]
PM_아름동

,연도,일시,측정소,PM2.5,Rad,Count_AWS,Cent_lat,Cent_lon,PM-Cent_d
0,0,01-01 00:00,아름동,0.044,0.15,4,36.509875,127.231625,0.015349
1,0,01-01 01:00,아름동,0.048,0.15,4,36.509875,127.231625,0.015349
2,0,01-01 02:00,아름동,0.060,0.15,4,36.509875,127.231625,0.015349
3,0,01-01 03:00,아름동,0.064,0.15,4,36.509875,127.231625,0.015349
4,0,01-01 04:00,아름동,0.064,0.15,4,36.509875,127.231625,0.015349
...,...,...,...,...,...,...,...,...,...
35059,3,12-31 19:00,아름동,0.028,0.15,4,36.509875,127.231625,0.015349
35060,3,12-31 20:00,아름동,0.024,0.15,4,36.509875,127.231625,0.015349
35061,3,12-31 21:00,아름동,0.028,0.15,4,36.509875,127.231625,0.015349
35062,3,12-31 22:00,아름동,0.048,0.15,4,36.509875,127.231625,0.015349


In [63]:
spec = loc_PM.loc['아름동'][['Rad','Count_AWS','Cent_lat','Cent_lon','PM-Cent_d']]
PM_아름동[['Rad','Count_AWS','Cent_lat','Cent_lon','PM-Cent_d']] = loc_PM.loc['아름동'][['Rad','Count_AWS','Cent_lat','Cent_lon','PM-Cent_d']]
PM_아름동

,연도,일시,측정소,PM2.5,Rad,Count_AWS,Cent_lat,Cent_lon,PM-Cent_d
0,0,01-01 00:00,아름동,0.044,0.15,4,36.509875,127.231625,0.015349
1,0,01-01 01:00,아름동,0.048,0.15,4,36.509875,127.231625,0.015349
2,0,01-01 02:00,아름동,0.060,0.15,4,36.509875,127.231625,0.015349
3,0,01-01 03:00,아름동,0.064,0.15,4,36.509875,127.231625,0.015349
4,0,01-01 04:00,아름동,0.064,0.15,4,36.509875,127.231625,0.015349
...,...,...,...,...,...,...,...,...,...
35059,3,12-31 19:00,아름동,0.028,0.15,4,36.509875,127.231625,0.015349
35060,3,12-31 20:00,아름동,0.024,0.15,4,36.509875,127.231625,0.015349
35061,3,12-31 21:00,아름동,0.028,0.15,4,36.509875,127.231625,0.015349
35062,3,12-31 22:00,아름동,0.048,0.15,4,36.509875,127.231625,0.015349


In [64]:
spec = loc_PM.loc['신흥동'][['Rad','Count_AWS','Cent_lat','Cent_lon','PM-Cent_d']]
PM_신흥동[['Rad','Count_AWS','Cent_lat','Cent_lon','PM-Cent_d']] = loc_PM.loc['아름동'][['Rad','Count_AWS','Cent_lat','Cent_lon','PM-Cent_d']]
PM_신흥동

,연도,일시,측정소,PM2.5,Rad,Count_AWS,Cent_lat,Cent_lon,PM-Cent_d
0,0,01-01 00:00,신흥동,0.044,0.15,4,36.509875,127.231625,0.015349
1,0,01-01 01:00,신흥동,0.064,0.15,4,36.509875,127.231625,0.015349
2,0,01-01 02:00,신흥동,0.060,0.15,4,36.509875,127.231625,0.015349
3,0,01-01 03:00,신흥동,0.072,0.15,4,36.509875,127.231625,0.015349
4,0,01-01 04:00,신흥동,0.080,0.15,4,36.509875,127.231625,0.015349
...,...,...,...,...,...,...,...,...,...
35059,3,12-31 19:00,신흥동,0.032,0.15,4,36.509875,127.231625,0.015349
35060,3,12-31 20:00,신흥동,0.052,0.15,4,36.509875,127.231625,0.015349
35061,3,12-31 21:00,신흥동,0.028,0.15,4,36.509875,127.231625,0.015349
35062,3,12-31 22:00,신흥동,0.024,0.15,4,36.509875,127.231625,0.015349


In [65]:
spec = loc_PM.loc['노은동'][['Rad','Count_AWS','Cent_lat','Cent_lon','PM-Cent_d']]
PM_노은동[['Rad','Count_AWS','Cent_lat','Cent_lon','PM-Cent_d']] = loc_PM.loc['아름동'][['Rad','Count_AWS','Cent_lat','Cent_lon','PM-Cent_d']]
PM_노은동

,연도,일시,측정소,PM2.5,Rad,Count_AWS,Cent_lat,Cent_lon,PM-Cent_d
0,0,01-01 00:00,노은동,0.052,0.15,4,36.509875,127.231625,0.015349
1,0,01-01 01:00,노은동,0.072,0.15,4,36.509875,127.231625,0.015349
2,0,01-01 02:00,노은동,0.068,0.15,4,36.509875,127.231625,0.015349
3,0,01-01 03:00,노은동,0.064,0.15,4,36.509875,127.231625,0.015349
4,0,01-01 04:00,노은동,0.064,0.15,4,36.509875,127.231625,0.015349
...,...,...,...,...,...,...,...,...,...
35059,3,12-31 19:00,노은동,0.032,0.15,4,36.509875,127.231625,0.015349
35060,3,12-31 20:00,노은동,0.048,0.15,4,36.509875,127.231625,0.015349
35061,3,12-31 21:00,노은동,0.020,0.15,4,36.509875,127.231625,0.015349
35062,3,12-31 22:00,노은동,0.056,0.15,4,36.509875,127.231625,0.015349


In [66]:
spec = loc_PM.loc['문창동'][['Rad','Count_AWS','Cent_lat','Cent_lon','PM-Cent_d']]
PM_문창동[['Rad','Count_AWS','Cent_lat','Cent_lon','PM-Cent_d']] = loc_PM.loc['아름동'][['Rad','Count_AWS','Cent_lat','Cent_lon','PM-Cent_d']]
PM_문창동

,연도,일시,측정소,PM2.5,Rad,Count_AWS,Cent_lat,Cent_lon,PM-Cent_d
0,0,01-01 00:00,문창동,0.112,0.15,4,36.509875,127.231625,0.015349
1,0,01-01 01:00,문창동,0.072,0.15,4,36.509875,127.231625,0.015349
2,0,01-01 02:00,문창동,0.084,0.15,4,36.509875,127.231625,0.015349
3,0,01-01 03:00,문창동,0.076,0.15,4,36.509875,127.231625,0.015349
4,0,01-01 04:00,문창동,0.064,0.15,4,36.509875,127.231625,0.015349
...,...,...,...,...,...,...,...,...,...
35059,3,12-31 19:00,문창동,0.056,0.15,4,36.509875,127.231625,0.015349
35060,3,12-31 20:00,문창동,0.048,0.15,4,36.509875,127.231625,0.015349
35061,3,12-31 21:00,문창동,0.036,0.15,4,36.509875,127.231625,0.015349
35062,3,12-31 22:00,문창동,0.036,0.15,4,36.509875,127.231625,0.015349
